# I] Import & options


In [1]:
import pandas, numpy

In [2]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'

In [3]:
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# II] GABARAP expression, ensembl. Data preparation 

In [4]:
%%time
# DL data GABARAP isoform expression
path = input_file_directory + "/xenabrower_brut_data/xena_GABARAP.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)
df_ori.head(3)

(19131, 11)
Wall time: 49.9 ms


,sample,samples,ENSG00000170296.9,ENST00000302386.9,ENST00000573928.1,ENST00000570856.1,ENST00000571253.1,ENST00000571129.5,ENST00000577035.5,_sample_type,_primary_site
0,TCGA-AJ-A2QM-01,TCGA-AJ-A2QM-01,11.99,7.026,10.130,9.047,6.293,2.015,11.1,Primary Tumor,Endometrium
1,GTEX-OIZI-0005-SM-2XCED,GTEX-OIZI-0005-SM-2XCED,11.27,7.823,9.316,7.894,3.403,-9.966,10.4,Normal Tissue,Blood
2,GTEX-XAJ8-0006-SM-46MVM,GTEX-XAJ8-0006-SM-46MVM,11.24,6.587,9.797,6.418,2.227,-9.966,10.4,Normal Tissue,Blood


In [5]:
# Column selection
df= df_ori[['sample',
            '_sample_type',
            '_primary_site',
            'ENSG00000170296.9',
            'ENST00000302386.9',
            'ENST00000573928.1', 
            'ENST00000577035.5']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df = df.assign(_primary_site=df['_primary_site'].str.title())

# Column rename
df = df.rename(columns = {'sample' : 'sample',
                          '_sample_type' : 'Sample_Type', 
                          '_primary_site' : 'Primary_Site',
                          'ENST00000302386.9': 'GABARAP_1',
                          'ENST00000573928.1': 'GABARAP_2',
                          'ENST00000577035.5': 'GABARAP_3',
                          'ENSG00000170296.9': 'GABARAP'})

# Calcul to have value from log2;  2**(GABARAP(1))-0.001
df = df.assign(GABARAP_total= pow(2,df['GABARAP'])-0.001,
          GABARAP_1tpm= pow(2,df['GABARAP_1'])-0.001,
          GABARAP_2tpm= pow(2,df['GABARAP_2'])-0.001,
          GABARAP_3tpm= pow(2,df['GABARAP_3'])-0.001)

# Calcul to have log2+1 from value; 
df = df.assign(log2_p1_GABARAP_total= numpy.log2(df['GABARAP_total']+1),
               log2_p1_GABARAP_1= numpy.log2(df['GABARAP_1tpm']+1),
               log2_p1_GABARAP_2= numpy.log2(df['GABARAP_2tpm']+1),
               log2_p1_GABARAP_3= numpy.log2(df['GABARAP_3tpm']+1))

# Calcul to see percentage of expression of the 3 isoforms of GABARAP 
df = df.assign(GABARAP_prot_tot= df['GABARAP_1tpm'] + df['GABARAP_2tpm'] + df['GABARAP_3tpm'])
df = df.assign(GABARAP_1perc= (df['GABARAP_1tpm'] *100)/ df['GABARAP_prot_tot'],
              GABARAP_2perc= (df['GABARAP_2tpm'] *100)/ df['GABARAP_prot_tot'],
              GABARAP_3perc= (df['GABARAP_3tpm'] *100)/ df['GABARAP_prot_tot'])                              

#Add columns, normalized on total
df = df.assign(GABARAP_1norm= df['GABARAP_1tpm']/df["GABARAP_total"],
               GABARAP_2norm= df['GABARAP_2tpm']/df["GABARAP_total"],
               GABARAP_3norm= df['GABARAP_3tpm']/df["GABARAP_total"])                                   

df.head()

,sample,Sample_Type,Primary_Site,GABARAP,GABARAP_1,GABARAP_2,GABARAP_3,GABARAP_total,GABARAP_1tpm,GABARAP_2tpm,...,log2_p1_GABARAP_1,log2_p1_GABARAP_2,log2_p1_GABARAP_3,GABARAP_prot_tot,GABARAP_1perc,GABARAP_2perc,GABARAP_3perc,GABARAP_1norm,GABARAP_2norm,GABARAP_3norm
0,TCGA-AJ-A2QM-01,Primary Tumor,Endometrium,11.99,7.026,10.130,11.10,4067.705861,130.326706,1120.555750,...,7.037017,10.131286,11.100656,3445.873507,3.782109,32.518772,63.699119,0.032039,0.275476,0.539614
1,GTEX-OIZI-0005-SM-2XCED,Normal Tissue,Blood,11.27,7.823,9.316,10.40,2469.493431,226.441351,637.374609,...,7.829351,9.318259,10.401066,2214.991061,10.223127,28.775493,61.001379,0.091695,0.258099,0.547147
2,GTEX-XAJ8-0006-SM-46MVM,Normal Tissue,Blood,11.24,6.587,9.797,10.40,2418.671971,96.134675,889.590998,...,6.601914,9.798619,10.401066,2336.900774,4.113768,38.067128,57.819105,0.039747,0.367801,0.558643
3,GTEX-ZP4G-0006-SM-4WWE6,Normal Tissue,Blood,11.18,6.138,9.749,10.36,2320.145197,70.423231,860.480282,...,6.158321,9.750674,10.361096,2245.130632,3.136710,38.326513,58.536777,0.030353,0.370873,0.566442
4,GTEX-WFG8-0006-SM-3GIKS,Normal Tissue,Blood,11.17,6.901,9.374,10.49,2304.118777,119.510033,663.520774,...,6.913009,9.376170,10.491002,2221.181360,5.380472,29.872427,64.747102,0.051868,0.287972,0.624165


In [6]:
#what kind of samples are the data ? 
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Control Analyte                                         1
Additional Metastatic                                   1
Post treatment Blood Cancer - Blood                     1
Name: Sample_T

In [7]:
#How many each organs are the data ? 
df['Primary_Site'].value_counts()

Brain                         1846
Lung                          1410
Breast                        1391
Skin                          1282
Kidney                        1193
Esophagus                      848
Prostate                       648
Colon                          639
Stomach                        624
Blood Vessel                   606
White Blood Cell               595
Thyroid Gland                  571
Head And Neck Region           564
Liver                          531
Ovary                          515
Adipose Tissue                 515
Blood                          444
Bladder                        435
Muscle                         396
Heart                          377
Pancreas                       350
Testis                         319
Cervix                         309
Thyroid                        279
Nerve                          278
Soft Tissue,Bone               264
Adrenal Gland                  205
Endometrium                    204
Paraganglia         

In [8]:
#Creation of two dataframe, Normal tissue and Primary tumor.
df_GABARAP_Normal = df[df['Sample_Type'].str.match('Normal Tissue')]
df_GABARAP_Primary = df[df['Sample_Type'].str.match('Primary Tumor')]

print('normal_tissue' , len(df_GABARAP_Normal))
print('primary_tumors' , len(df_GABARAP_Primary))

normal_tissue 7429
primary_tumors 9185


In [9]:
# create df with only Normal and Primary 
df_GABARAP_NormPrim = pandas.concat([df_GABARAP_Normal, df_GABARAP_Primary], axis=0)
df_GABARAP_NormPrim.reset_index(drop=True, inplace=True)
df_GABARAP_NormPrim.head()

,sample,Sample_Type,Primary_Site,GABARAP,GABARAP_1,GABARAP_2,GABARAP_3,GABARAP_total,GABARAP_1tpm,GABARAP_2tpm,...,log2_p1_GABARAP_1,log2_p1_GABARAP_2,log2_p1_GABARAP_3,GABARAP_prot_tot,GABARAP_1perc,GABARAP_2perc,GABARAP_3perc,GABARAP_1norm,GABARAP_2norm,GABARAP_3norm
0,GTEX-OIZI-0005-SM-2XCED,Normal Tissue,Blood,11.27,7.823,9.316,10.40,2469.493431,226.441351,637.374609,...,7.829351,9.318259,10.401066,2214.991061,10.223127,28.775493,61.001379,0.091695,0.258099,0.547147
1,GTEX-XAJ8-0006-SM-46MVM,Normal Tissue,Blood,11.24,6.587,9.797,10.40,2418.671971,96.134675,889.590998,...,6.601914,9.798619,10.401066,2336.900774,4.113768,38.067128,57.819105,0.039747,0.367801,0.558643
2,GTEX-ZP4G-0006-SM-4WWE6,Normal Tissue,Blood,11.18,6.138,9.749,10.36,2320.145197,70.423231,860.480282,...,6.158321,9.750674,10.361096,2245.130632,3.136710,38.326513,58.536777,0.030353,0.370873,0.566442
3,GTEX-WFG8-0006-SM-3GIKS,Normal Tissue,Blood,11.17,6.901,9.374,10.49,2304.118777,119.510033,663.520774,...,6.913009,9.376170,10.491002,2221.181360,5.380472,29.872427,64.747102,0.051868,0.287972,0.624165
4,GTEX-QCQG-0006-SM-5SI8M,Normal Tissue,Blood,11.08,6.793,9.684,10.15,2164.771947,110.890118,822.572053,...,6.805939,9.685751,10.151268,2069.660310,5.357890,39.744302,54.897808,0.051225,0.379981,0.524858


In [10]:
#save to csv
path = 'GABARAP/dataframes_for_input/'
df_GABARAP_NormPrim.to_csv(output_file_directory+path+'df_GABARAP_NormPrim.csv',sep = ",", index=False)